## 추가 변수 생성 및 결측치 처리

In [14]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

In [15]:
## total_busdata 
total_busdata = pd.read_csv(
    filepath_or_buffer="../data/total_busdata.csv"
    )

### 첫차, 막차시간 변수 결측값 처리

In [16]:
# 시간 변환 함수 정의

# to minutes
def convert_to_minutes(time):
    if pd.isna(time):  # NaN 값인지 확인
        return time  # NaN 값을 그대로 반환
    hours = int(time // 100)
    minutes = int(time % 100)
    return hours * 60 + minutes

# to hours
def convert_to_hours_minutes(minutes):
    hours = int(minutes // 60)
    remaining_minutes = int(minutes % 60)
    return hours * 100 + remaining_minutes


In [17]:
# startvehicletime 과 endvehicletime 변수의 시간 변환 값을 각각 start, end 변수에 넣음

total_busdata['start'] = total_busdata['startvehicletime'].apply(convert_to_minutes)
total_busdata['end'] = total_busdata['endvehicletime'].apply(convert_to_minutes)

In [18]:
# start, end 열의 평균 
mean_start = total_busdata['start'].mean()
mean_end = total_busdata['end'].mean()

# start, end 열의 결측값에 평균 대입
total_busdata['start'] = total_busdata['start'].fillna(mean_start)
total_busdata['end'] = total_busdata['end'].fillna(mean_end)

# start, end 열의 평균을 원래 시간 형태로 변환
mean_start = convert_to_hours_minutes(mean_start)
mean_end = convert_to_hours_minutes(mean_end)

In [19]:
# startvehicletime과 endvehicletime의 결측값에 대입
total_busdata['startvehicletime'] = total_busdata['startvehicletime'].fillna(mean_start)
total_busdata['endvehicletime'] = total_busdata['endvehicletime'].fillna(mean_end)

### 배차간격 결측값 처리

In [20]:
# 배차간격 결측값 : 시,군 전체 평균을 결측값에 넣음
intervaltime_avg = total_busdata['intervaltime'].mean()
total_busdata['intervaltime'] = total_busdata['intervaltime'].fillna(intervaltime_avg)

intervalsattime_avg = total_busdata['intervalsattime'].mean()
total_busdata['intervalsattime'] = total_busdata['intervalsattime'].fillna(intervalsattime_avg)

intervalsuntime_avg = total_busdata['intervalsuntime'].mean()
total_busdata['intervalsuntime'] = total_busdata['intervalsuntime'].fillna(intervalsuntime_avg)

total_busdata[['intervaltime','intervalsattime','intervalsuntime']]= total_busdata[['intervaltime','intervalsattime','intervalsuntime']].round(1)

### 버스대수 변수 생성

In [21]:
# 총 운행 시간 계산 (분 단위)
total_busdata['total_operating_time'] = total_busdata['end'] - total_busdata['start']

# 평균 배차 간격 변수
total_busdata['interval_avg'] = (total_busdata['intervaltime']*5+total_busdata['intervalsattime']+total_busdata['intervalsuntime']) / 7

# 필요한 버스 대수 
total_busdata['num_bus'] = total_busdata['total_operating_time'] / total_busdata['interval_avg']
total_busdata['num_bus'] = total_busdata['num_bus'].astype(int)


### 굴곡도 결측치 처리

In [22]:
curve_avg = total_busdata['굴곡도'].mean()
total_busdata['굴곡도'] = total_busdata['굴곡도'].fillna(curve_avg)
total_busdata['굴곡도'] = total_busdata['굴곡도'].round(1)

### 노선정류장수 결측치 처리 (시, 군별로)

In [23]:
## 함양군 <- 진주시
jj_mean = total_busdata[total_busdata["city_name"] == "진주시"]["노선거리(km)"].mean()
hy_index = total_busdata[total_busdata["city_name"] == "함양군"]["노선거리(km)"].index
for i in hy_index:
    total_busdata.loc[i, "노선거리(km)"] = jj_mean

## 고성군 <- 창녕군
cn_mean = total_busdata[total_busdata["city_name"] == "창녕군"]["노선거리(km)"].mean()
gs_index = total_busdata[total_busdata["city_name"] == "고성군"]["노선거리(km)"].index
for i in gs_index:
    total_busdata.loc[i, "노선거리(km)"] = cn_mean

## 남해군 <- 통영시
ty_mean = total_busdata[total_busdata["city_name"] == "통영시"]["노선거리(km)"].mean()
nh_index = total_busdata[total_busdata["city_name"] == "남해군"]["노선거리(km)"].index
for i in nh_index:
    total_busdata.loc[i, "노선거리(km)"] = ty_mean

In [24]:
total_busdata['노선정류장수'] = total_busdata.groupby('city_name')['노선정류장수'].transform(lambda x: x.fillna(x.mean()))
total_busdata['노선거리(km)'] = total_busdata.groupby('city_name')['노선거리(km)'].transform(lambda x: x.fillna(x.mean()))


total_busdata['노선정류장수'] = total_busdata['노선정류장수'].astype(int)
total_busdata['노선거리(km)'] = total_busdata['노선거리(km)']

In [25]:
total_busdata.drop(columns=['usage']).isnull().sum()

city_name               0
routeno                 0
routetp                 0
startnodenm             0
endnodenm               0
startvehicletime        0
endvehicletime          0
intervaltime            0
intervalsattime         0
intervalsuntime         0
노선정류장수                  0
노선거리(km)                0
굴곡도                     0
start                   0
end                     0
total_operating_time    0
interval_avg            0
num_bus                 0
dtype: int64

In [26]:
total_busdata.to_csv("../data/여진_total_busdata.csv", index=False)

#### 결측치 종류

1. Missing completely at random (MCAR) - 완전 무작위 결측
결측값의 발생이 다른 변수와 상관이 없는 경우
ex) 전산오류, 통신문제 등으로 데이터 누락

2. Missing at random (MAR) - 무작위 결측
결측값의 발생이 특정 변수와 관련이 있으나 얻고자 하는 결과와는 상관이 없는 경우

3. Not missing at random (NMAR) - 비무작위 결측
결측값 발생이 다른 변수와 상관이 있는 경우



#### 결측치 대체 방법

1. 아무것도 하지 않기

2. 데이터 제거 

3. 중앙값, 평균값, 최빈값, 0, 상수값으로 대체

4. K-NN(K-Nearest Neighbor) 대체 
각 데이터 포인트의 가장 가까운 k개의 이웃을 찾아서 결측값을 예측하는 방식
유클리드 거리를 사용하여 각 노선의 길이, 정류장 수, 첫차 시간, 막차 시간 등 모든 특징을 포함한 거리 계산을 통해 결측값을 대체
-> 결측값이 있는 노선과 가장 유사한 3개의 노선을 찾아 그들의 값을 기반으로 결측값을 예측 

<1>
#from impyute.imputation.cs import fast_knn

#np_imputed = fast_knn(df.values, k=5)
#df_imputed = pd.DataFrame(np_imputed)

<2>

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

#데이터 정규화
scaler = MinMaxScaler()
total_busdata_scaled = scaler.fit_transform(total_busdata)

#KNNImputer 생성 (k=3 사용)
imputer = KNNImputer(n_neighbors=3)

#결측값 대체
total_busdata_filled = imputer.fit_transform(total_busdata_scaled)

#정규화된 데이터를 원래 스케일로 되돌림
total_busdata_filled = scaler.inverse_transform(total_busdata_filled)

#대체된 값을 원래 데이터프레임에 반영
total_busdata['usage'] = total_busdata_filled[:, 0]


5. MICE(Multivariate Imputation by Chained Equation)

#from impyute.imputation.cs import mice

#np_imputed=mice(df.values)
#df_imputed = pd.DataFrame(np_immputed)

6. 딥러닝을 이용한 Imputation (범주형이나 숫자가 아닌 자료형에 효과적)
#import datawig

#imputer = datawig.SimpleImputer(
#input_columns = ['X_1', 'X_2', 'X_3'], # impute에 사용할 col 지정
#output_column = 'X_5') # 컬럼 X_5의 결측치를 채운다

#imputer.fit(train_df=df_null, num_epochs=50)

#df_null_only = df_null[df_null['X_5'].isnull()]

#np_imputed = imputer.predict(df_null_only)
#df_imputed = pd.DataFrame(np_imputed)

